In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, SpatialDropout1D
from keras.utils import  to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
data = pd.read_csv('./Data/2018-06-06-ss.cleaned.csv')
data = data.iloc[:20000]

sequences = data.iloc[:,2]
structures = data.iloc[:,4]
data = []


In [4]:

#structures = structures_to_int(structures)
label_encoder = LabelEncoder()
label_encoder.fit(structures)
labels = label_encoder.transform(structures)

# Pad sequences with zeros, and make X a 3D vector
max_length = 3000
X = []
for i in range (len(sequences)):
    sequences[i] = sequences[i].ljust(max_length, '0')
    X.append(list(sequences[i]))

X = np.array(X)
#print(X[:1])

# Convert X to integer-encoded data
char_set = sorted(set(''.join(sequences)))
char_to_int = dict((c, i+1) for i, c in enumerate(char_set))
X_int = []
for i in range(len(X)):
    seq_int = [char_to_int[char] for char in X[i]]
    X_int.append(seq_int)
X_int = np.array(X_int)

# Convert X_int to float and normalize the values
X_float = X_int.astype(float)

# Convert integer labels to categorical labels
Y = to_categorical(labels)

print(X_float[:3])

[[ 6.  5. 12. ...  2.  2.  2.]
 [11.  4. 11. ...  2.  2.  2.]
 [11.  3. 11. ...  2.  2.  2.]]


In [6]:
# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim = max_length+1, output_dim= 128, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_float, Y, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
 67/125 [===============>..............] - ETA: 22:42 - loss: 7.3680 - accuracy: 0.0389